# Problem set 4: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [1]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
### nltk.download('averaged_perceptron_tagger')
### nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
### ! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

## 0.1 Load and clean text data

In [2]:
## first, unzip the file pset4_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [3]:
## your code to subset to one press release and take the string
kickback_row = doj[doj["id"] == "17-1204"]
kickback_row

pharma = str(kickback_row["contents"])
print(type(pharma))

id  \
4909  17-1204   

                                                                                         title  \
4909  Founder and Owner of Pharmaceutical Company Insys Arrested and Charged with Racketeering   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

<class 'str'>


### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (so can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp
- `processtext` function here has an example of tokenizing and filtering to words where `.isalpha()` is true: https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/05_textasdata_partII_topicmodeling_solutions.ipynb
- Part of speech tagging section of this code: https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/05_textasdata_partI_textmining_solutions.ipynb



In [4]:
## your code here to restrict to alpha
pharma_alpha = re.sub(r'\W+', ' ', pharma)
pharma_alpha = re.sub(r'\d+', ' ', pharma_alpha)

In [5]:
## your code here for part of speech tagging

pharma_tokens = word_tokenize(pharma_alpha)
pharma_pos = pos_tag(pharma_tokens)

pharma_df = pd.DataFrame.from_records(pharma_pos, columns = ["word", "pos"])
pharma_adj_df = pharma_df[pharma_df["pos"].str.contains("JJ") == True]

pharma_adj_count = pharma_adj_df.value_counts("word").sort_values(ascending = False)

pharma_adj_count.head()

word
former        8
opioid        5
nationwide    4
other         3
addictive     3
dtype: int64

## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

**Resources**:

- For parts A and B: named entity recognition part of this code: https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/05_textasdata_partI_textmining_solutions.ipynb

In [6]:
## your code here for part A
spacy_pharma = nlp(pharma)

In [7]:
## your code here for part B

for one_tok in spacy_pharma.ents:
    print(one_tok)
    print("Entity: " + one_tok.text + "; NER tag: " + one_tok.label_)
    #if one_tok.label_ == "LAW":
    #    print("Entity: " + one_tok.text + "; NER tag: " + one_tok.label_)

4909
Entity: 4909; NER tag: DATE
Insys Therapeutics Inc.
Entity: Insys Therapeutics Inc.; NER tag: ORG
today
Entity: today; NER tag: DATE
Fentanyl
Entity: Fentanyl; NER tag: PERSON
More than 20,000
Entity: More than 20,000; NER tag: CARDINAL
Americans
Entity: Americans; NER tag: NORP
last year
Entity: last year; NER tag: DATE
millions
Entity: millions; NER tag: CARDINAL
Jeff Sessions
Entity: Jeff Sessions; NER tag: PERSON
This Justice Department
Entity: This Justice Department; NER tag: ORG
Trump
Entity: Trump; NER tag: PERSON
American
Entity: American; NER tag: NORP
”John N. Kapoor
Entity: ”John N. Kapoor; NER tag: PERSON
74
Entity: 74; NER tag: DATE
Phoenix
Entity: Phoenix; NER tag: GPE
Ariz.
Entity: Ariz.; NER tag: GPE
the Board of Directors
Entity: the Board of Directors; NER tag: ORG
Insys
Entity: Insys; NER tag: ORG
this morning
Entity: this morning; NER tag: TIME
Arizona
Entity: Arizona; NER tag: GPE
RICO
Entity: RICO; NER tag: LAW
Kapoor
Entity: Kapoor; NER tag: PERSON
Executiv

C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

In [8]:
## your code here 

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [9]:
## your code here
maybe_sentence = []
maybe_sentence_years = []
for one_tok in spacy_pharma.ents:
    if one_tok.label_ == "DATE":
        maybe_sentence.append(one_tok.text)

years = [re.findall(r'\w+\syears*', s) for s in maybe_sentence]

for r in years:
    if r != []:
        maybe_sentence_years = maybe_sentence_years + r
        
print(maybe_sentence)
print(maybe_sentence_years)

['4909', 'today', 'last year', '74', 'today', 'a later date', 'today', 'December 2016.The', '40', '42', '46', '36', '43', '53', 'Today', 'Today', '20 years', 'three years', 'five years', 'three years']
['last year', '20 years', 'three years', 'five years', 'three years']


E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, `re.search` and `re.findall` examples here for filtering to ones containing year (multiple approaches; some need not involve `re`): https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/05_basicregex_solutions.ipynb

In [10]:
## your code here


## 1.3 sentiment analysis  (10 points)

- Sentiment analysis section of this script: https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/05_textasdata_partI_textmining_solutions.ipynb


A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [11]:
## your code here for subsetting
doj_subset = doj[doj["topics_clean"] == "Civil Rights"]
doj_subset = doj_subset.append(doj[doj["topics_clean"] == "Hate Crimes"])
doj_subset = doj_subset.append(doj[doj["topics_clean"] == "Project Safe Childhood"]).reset_index(drop = True)
doj_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                707 non-null    object
 1   title             717 non-null    object
 2   contents          717 non-null    object
 3   date              717 non-null    object
 4   topics_clean      717 non-null    object
 5   components_clean  717 non-null    object
dtypes: object(6)
memory usage: 33.7+ KB


B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- I used a function + list comprehension to execute and it takes about 30 seconds on my local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [19]:
## your code here to define function
test_string = "A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened.    James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack.   Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up.   “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division.  “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.”   “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.”    This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section."

def sentiment_analysis(string):
    #new_string = string
    #
    #spacy_string = nlp(string)
    #entities = []
    #for one_tok in spacy_string.ents:
    #    entities.append(one_tok.text)
    #
    #for ent in entities:
    #    print(ent)
    #    print(type(ent))
    #    new_string.replace(ent, "")
    
    #string
    #new_string
        
    sent_obj = SentimentIntensityAnalyzer()
    sentiment = sent_obj.polarity_scores(string)
    sentiment
    (neg, neu, pos, comp) = sentiment["neg"], sentiment["neu"], sentiment["pos"], sentiment["compound"]

    return neg, neu, pos, comp

sentiment_analysis(test_string)


(0.169, 0.763, 0.068, -0.9893)

In [20]:
## your code here executing the function
releases_list = doj_subset["contents"].to_list()

for release in releases_list:
    print(sentiment_analysis(release))
    

(0.169, 0.763, 0.068, -0.9893)
(0.146, 0.774, 0.079, -0.9814)
(0.097, 0.751, 0.152, 0.9623)
(0.131, 0.737, 0.132, -0.6771)
(0.215, 0.633, 0.152, -0.7717)
(0.029, 0.857, 0.114, 0.9485)
(0.103, 0.816, 0.082, -0.7783)
(0.109, 0.834, 0.057, -0.9746)
(0.08, 0.849, 0.071, -0.2575)
(0.057, 0.854, 0.089, 0.9493)
(0.105, 0.83, 0.065, -0.985)
(0.118, 0.801, 0.081, -0.9578)
(0.0, 0.834, 0.166, 0.9949)
(0.097, 0.858, 0.044, -0.91)
(0.079, 0.863, 0.058, -0.6085)
(0.044, 0.821, 0.134, 0.9897)
(0.072, 0.816, 0.112, 0.9673)
(0.033, 0.842, 0.126, 0.9893)
(0.069, 0.843, 0.088, 0.8779)
(0.105, 0.734, 0.16, 0.9872)
(0.056, 0.839, 0.105, 0.9833)
(0.008, 0.828, 0.164, 0.9976)
(0.021, 0.864, 0.114, 0.9963)
(0.021, 0.828, 0.151, 0.9959)
(0.069, 0.821, 0.11, 0.9838)
(0.079, 0.812, 0.109, 0.9636)
(0.062, 0.797, 0.141, 0.9962)
(0.027, 0.913, 0.06, 0.9147)
(0.076, 0.789, 0.135, 0.9918)
(0.137, 0.766, 0.097, -0.9816)
(0.118, 0.79, 0.092, -0.9855)
(0.115, 0.828, 0.057, -0.9995)
(0.109, 0.797, 0.093, -0.9371)
(0.172

(0.029, 0.875, 0.096, 0.9028)
(0.125, 0.77, 0.106, -0.9186)
(0.0, 0.879, 0.121, 0.8779)
(0.0, 0.84, 0.16, 0.9442)
(0.026, 0.74, 0.234, 0.9897)
(0.0, 0.916, 0.084, 0.9118)
(0.196, 0.735, 0.069, -0.969)
(0.133, 0.787, 0.079, -0.9745)
(0.137, 0.777, 0.086, -0.9778)
(0.178, 0.75, 0.072, -0.9936)
(0.022, 0.863, 0.115, 0.9861)
(0.195, 0.783, 0.022, -0.9959)
(0.135, 0.797, 0.068, -0.9571)
(0.149, 0.817, 0.034, -0.9875)
(0.259, 0.713, 0.028, -0.9968)
(0.167, 0.748, 0.086, -0.9863)
(0.235, 0.712, 0.052, -0.9963)
(0.211, 0.695, 0.094, -0.9817)
(0.17, 0.739, 0.092, -0.9799)
(0.05, 0.858, 0.092, 0.9545)
(0.047, 0.819, 0.134, 0.994)
(0.163, 0.737, 0.1, -0.9976)
(0.148, 0.762, 0.09, -0.9633)
(0.013, 0.856, 0.131, 0.9986)
(0.031, 0.831, 0.138, 0.9947)
(0.125, 0.746, 0.13, -0.2054)
(0.141, 0.803, 0.056, -0.9786)
(0.122, 0.819, 0.059, -0.9798)
(0.082, 0.844, 0.073, -0.3612)
(0.179, 0.772, 0.049, -0.9965)
(0.257, 0.699, 0.044, -0.9943)
(0.104, 0.818, 0.078, -0.9081)
(0.205, 0.739, 0.056, -0.9983)
(0.117

(0.098, 0.81, 0.092, -0.3182)
(0.081, 0.829, 0.09, 0.5267)
(0.144, 0.756, 0.1, -0.9864)
(0.081, 0.815, 0.104, 0.8225)
(0.064, 0.812, 0.124, 0.9979)
(0.102, 0.777, 0.12, 0.5423)
(0.105, 0.79, 0.105, -0.1717)
(0.098, 0.814, 0.088, -0.8316)
(0.096, 0.768, 0.136, 0.9022)
(0.085, 0.805, 0.11, 0.7845)
(0.108, 0.768, 0.124, 0.7128)
(0.121, 0.782, 0.096, -0.802)
(0.133, 0.772, 0.095, -0.9446)
(0.086, 0.8, 0.114, 0.8481)
(0.152, 0.733, 0.114, -0.9477)
(0.13, 0.763, 0.107, -0.6597)
(0.108, 0.775, 0.117, 0.8173)
(0.102, 0.795, 0.103, -0.6909)
(0.127, 0.788, 0.085, -0.9169)
(0.092, 0.823, 0.085, -0.7411)
(0.101, 0.805, 0.095, -0.34)
(0.068, 0.814, 0.118, 0.9485)
(0.089, 0.868, 0.043, -0.9741)
(0.078, 0.813, 0.109, 0.9062)
(0.117, 0.794, 0.089, -0.875)
(0.132, 0.758, 0.11, -0.6486)
(0.13, 0.769, 0.101, -0.9738)
(0.14, 0.798, 0.062, -0.9952)
(0.135, 0.798, 0.067, -0.9726)
(0.095, 0.783, 0.122, 0.8332)
(0.124, 0.772, 0.104, -0.9423)
(0.104, 0.76, 0.136, 0.743)
(0.096, 0.795, 0.109, 0.4767)
(0.121, 0.

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [46]:
## your code here

#for i, row in df.iterrows():
#    ifor_val = something
#    if <condition>:
#        ifor_val = something_else
#    df.at[i,'ifor'] = ifor_val

doj_subset_wscore = doj_subset

doj_subset_wscore["sent_neg"]  = 0
doj_subset_wscore["sent_neu"]  = 0
doj_subset_wscore["sent_pos"]  = 0
doj_subset_wscore["sent_comp"] = 0

#Used StackOverflow to troubleshoot this chunk of code
for i, row in doj_subset.iterrows():
    (neg, neu, pos, comp) = sentiment_analysis(doj_subset_wscore.at[i, "contents"])
    print([neg, neu, pos, comp])
    doj_subset_wscore.at[i, "sent_neg"]  = neg
    doj_subset_wscore.at[i, "sent_neu"]  = neu
    doj_subset_wscore.at[i, "sent_pos"]  = pos
    doj_subset_wscore.at[i, "sent_comp"] = comp
    
doj_subset_wscore

[0.169, 0.763, 0.068, -0.9893]
[0.146, 0.774, 0.079, -0.9814]
[0.097, 0.751, 0.152, 0.9623]
[0.131, 0.737, 0.132, -0.6771]
[0.215, 0.633, 0.152, -0.7717]
[0.029, 0.857, 0.114, 0.9485]
[0.103, 0.816, 0.082, -0.7783]
[0.109, 0.834, 0.057, -0.9746]
[0.08, 0.849, 0.071, -0.2575]
[0.057, 0.854, 0.089, 0.9493]
[0.105, 0.83, 0.065, -0.985]
[0.118, 0.801, 0.081, -0.9578]
[0.0, 0.834, 0.166, 0.9949]
[0.097, 0.858, 0.044, -0.91]
[0.079, 0.863, 0.058, -0.6085]
[0.044, 0.821, 0.134, 0.9897]
[0.072, 0.816, 0.112, 0.9673]
[0.033, 0.842, 0.126, 0.9893]
[0.069, 0.843, 0.088, 0.8779]
[0.105, 0.734, 0.16, 0.9872]
[0.056, 0.839, 0.105, 0.9833]
[0.008, 0.828, 0.164, 0.9976]
[0.021, 0.864, 0.114, 0.9963]
[0.021, 0.828, 0.151, 0.9959]
[0.069, 0.821, 0.11, 0.9838]
[0.079, 0.812, 0.109, 0.9636]
[0.062, 0.797, 0.141, 0.9962]
[0.027, 0.913, 0.06, 0.9147]
[0.076, 0.789, 0.135, 0.9918]
[0.137, 0.766, 0.097, -0.9816]
[0.118, 0.79, 0.092, -0.9855]
[0.115, 0.828, 0.057, -0.9995]
[0.109, 0.797, 0.093, -0.9371]
[0.172

[0.018, 0.86, 0.122, 0.995]
[0.029, 0.875, 0.096, 0.9028]
[0.125, 0.77, 0.106, -0.9186]
[0.0, 0.879, 0.121, 0.8779]
[0.0, 0.84, 0.16, 0.9442]
[0.026, 0.74, 0.234, 0.9897]
[0.0, 0.916, 0.084, 0.9118]
[0.196, 0.735, 0.069, -0.969]
[0.133, 0.787, 0.079, -0.9745]
[0.137, 0.777, 0.086, -0.9778]
[0.178, 0.75, 0.072, -0.9936]
[0.022, 0.863, 0.115, 0.9861]
[0.195, 0.783, 0.022, -0.9959]
[0.135, 0.797, 0.068, -0.9571]
[0.149, 0.817, 0.034, -0.9875]
[0.259, 0.713, 0.028, -0.9968]
[0.167, 0.748, 0.086, -0.9863]
[0.235, 0.712, 0.052, -0.9963]
[0.211, 0.695, 0.094, -0.9817]
[0.17, 0.739, 0.092, -0.9799]
[0.05, 0.858, 0.092, 0.9545]
[0.047, 0.819, 0.134, 0.994]
[0.163, 0.737, 0.1, -0.9976]
[0.148, 0.762, 0.09, -0.9633]
[0.013, 0.856, 0.131, 0.9986]
[0.031, 0.831, 0.138, 0.9947]
[0.125, 0.746, 0.13, -0.2054]
[0.141, 0.803, 0.056, -0.9786]
[0.122, 0.819, 0.059, -0.9798]
[0.082, 0.844, 0.073, -0.3612]
[0.179, 0.772, 0.049, -0.9965]
[0.257, 0.699, 0.044, -0.9943]
[0.104, 0.818, 0.078, -0.9081]
[0.205, 0

[0.098, 0.81, 0.092, -0.3182]
[0.081, 0.829, 0.09, 0.5267]
[0.144, 0.756, 0.1, -0.9864]
[0.081, 0.815, 0.104, 0.8225]
[0.064, 0.812, 0.124, 0.9979]
[0.102, 0.777, 0.12, 0.5423]
[0.105, 0.79, 0.105, -0.1717]
[0.098, 0.814, 0.088, -0.8316]
[0.096, 0.768, 0.136, 0.9022]
[0.085, 0.805, 0.11, 0.7845]
[0.108, 0.768, 0.124, 0.7128]
[0.121, 0.782, 0.096, -0.802]
[0.133, 0.772, 0.095, -0.9446]
[0.086, 0.8, 0.114, 0.8481]
[0.152, 0.733, 0.114, -0.9477]
[0.13, 0.763, 0.107, -0.6597]
[0.108, 0.775, 0.117, 0.8173]
[0.102, 0.795, 0.103, -0.6909]
[0.127, 0.788, 0.085, -0.9169]
[0.092, 0.823, 0.085, -0.7411]
[0.101, 0.805, 0.095, -0.34]
[0.068, 0.814, 0.118, 0.9485]
[0.089, 0.868, 0.043, -0.9741]
[0.078, 0.813, 0.109, 0.9062]
[0.117, 0.794, 0.089, -0.875]
[0.132, 0.758, 0.11, -0.6486]
[0.13, 0.769, 0.101, -0.9738]
[0.14, 0.798, 0.062, -0.9952]
[0.135, 0.798, 0.067, -0.9726]
[0.095, 0.783, 0.122, 0.8332]
[0.124, 0.772, 0.104, -0.9423]
[0.104, 0.76, 0.136, 0.743]
[0.096, 0.795, 0.109, 0.4767]
[0.121, 0.

,id,title,contents,date,topics_clean,components_clean,sent_neg,sent_neu,sent_pos,sent_comp
0,17-1235,Additional Former Correctional Officer Pleads Guilty to Beating of Handcuffed and Shackled Inmate at Angola State Prison,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",2017-11-02T00:00:00-04:00,Civil Rights,"Civil Rights Division; USAO - Louisiana, Middle",0.169,0.763,0.068,-0.9893
1,17-240,Anoka County Resident Sentenced to Six Months in Prison for Threatening Two Clinics that Provide Reproductive Health Services,"On, Feb. 27, 2017, Michael John Harris, 34, was sentenced to six months imprisonment and one year of supervised release for making telephonic threats to two medical clinics in Minneapolis, Minnesota, that provide reproductive health services. On March 2, 2016, Harris pleaded guilty to two violations of 18 U.S.C. § 248(a)(1). During his plea hearing, Harris admitted that on May 12, 2014, he made telephonic threats to two different health clinics in Minneapolis that provide reproductive health services. In a call to the first clinic, Harris threatened to kill the recipient of the call with his bare hands and to cut the recipient’s head off with a band saw. In a call to the second clinic, Harris told the recipient that he was going to kill the recipient and the recipient’s co-workers, and that he was going to travel to the clinic and shoot everyone present. Harris further admitted that he made these threats because the recipient was and has been, and in order to intimidate the recipient and any other person from, obtaining and providing reproductive health services. “This defendant threatened these clinic workers with death and brutality,” said Acting Assistant Attorney General Tom Wheeler of the Justice Department’s Civil Rights Division. “The department is pleased that the defendant accepted responsibility and will face consequences for his actions. The Department is committed to vigorously enforcing the civil rights of all individuals in this country.” “The violence threatened by this d

D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [28]:
## agg and find the mean compound score by topic
comp_mean = doj_subset.groupby("topics_clean").agg({"sent_comp": "mean"})
comp_mean

,sent_comp
topics_clean,
Civil Rights,0.154595
Hate Crimes,-0.884882
Project Safe Childhood,-0.245364


# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/
- Here's code with topic modeling steps: https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/05_textasdata_partII_topicmodeling_solutions.ipynb

In [29]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [35]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/spencer/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [44]:
## your code defining a text processing function
def preprocessing(string):
    default_stopwords = stopwords.words("english")
    total_stopwords = default_stopwords + custom_doj_stopwords
    
    lower = string.lower()
    wordpunct_tokenize(string)
    string_nostop = [word
                     for word in wordpunct_tokenize(string)
                     if word not in total_stopwords]
    
    string_alpha = [word
                    for word in string_nostop
                    if word.isalpha() == True]
    
    string_bigwords = [word
                       for word in string_alpha
                       if len(word) >= 4]
    
    string_stemmed = []
    snow_stemmer = SnowballStemmer(language='english')
    for word in string_bigwords:
        x = snow_stemmer.stem(word)
        string_stemmed.append(x)
    
    string_clean = ""
    for word in string_stemmed:
        string_clean = string_clean + word + " "
    
    return string_clean

In [49]:
## your code executing the function
doj_subset_wscore["processed_text"] = doj_subset_wscore["contents"].apply(preprocessing)
doj_subset_wscore

,id,title,contents,date,topics_clean,components_clean,sent_neg,sent_neu,sent_pos,sent_comp,processed_text
0,17-1235,Additional Former Correctional Officer Pleads Guilty to Beating of Handcuffed and Shackled Inmate at Angola State Prison,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",2017-11-02T00:00:00-04:00,Civil Rights,"Civil Rights Division; USAO - Louisiana, Middle",0.169,0.763,0.068,-0.9893,former supervisori correct offic louisiana state penitentiari angola louisiana plead guilti yesterday connect beat handcuf shackl inmat addit conspir cover misconduct falsifi offici record lie intern investig happen jame savoy marksvill louisiana admit plea hear wit offic use excess forc inmat fail interven conspir offic cover beat engag varieti obstruct act person falsifi offici prison record cover attack scotti kennedi beeb arkansa john sander marksvill louisiana previous plead guilti novemb septemb role beat cover everi citizen right process protect unreason forc correct offic violat basic constitut must held account egregi action said act assist attorney general john gore civil right divis justic depart continu vigor prosecut correct offic violat public trust commit crime cover violat feder crimin yesterday anoth exampl unwav commit pursu violat feder crimin law said act unit state attorney middl district louisiana corey amundson continu work close justic depart civil right divis ensur this investig baton roug resid agenc prosecut assist attorney frederick menner middl district louisiana trial attorney christoph perra civil right divis crimin section
1,17-240,Anoka County Resident Sentenced to Six Months in Prison for Threatening Two Clinics that Provide Reproductive Health Services,"On, Feb. 27, 2017, Michael John Harris, 34, was sentenced to six months imprisonment and one year of supervised release for making telephonic threats to two medical clinics in Minneapolis, Minnesota, that provide reproductive health services. On March 2, 2016, Harris pleaded guilty to two violations of 18 U.S.C. § 248(a)(

In [66]:
## your code showing the examples
doj_subset_wscore_print = doj_subset_wscore[doj_subset_wscore["id"] == "16-718"]
doj_subset_wscore_print = doj_subset_wscore_print.append(doj_subset_wscore[doj_subset_wscore["id"] == "16-217"])
doj_subset_wscore_print = doj_subset_wscore_print[["id", "contents", "processed_text"]]
doj_subset_wscore_print

$$$
$$$
$$$
$$$


,id,contents,processed_text
293,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",nine count indict unseal today mississippi correct offic charg beat inmat third charg help cover indict charg lawardrick marsher robert sturdiv offic mississippi state penitentiari parchman mississippi beat includ kick punch throw victim ground marsher sturdiv charg violat right convict prison free cruel unusu punish sturdiv also charg fail interven marsher punch beat indict alleg action involv danger weapon result bodili injuri victim third offic deont pate charg along marsher sturdiv conspir cover beat indict alleg three offic submit fals report three lie convict marsher sturdiv face maximum sentenc year prison excess forc charg each three offic face five year prison conspiraci fals statement charg year prison fals report charg indict mere accus defend presum innoc unless proven guilti this investig jackson divis cooper mississippi depart correct prosecut assist attorney robert coleman northern district mississippi trial attorney dana mulhaus civil right divis crimin section marsher indict
164,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and s

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.

**Resources**:

- Here contains an example of applying the `create_dtm` function: https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/05_textasdata_partII_topicmodeling_solutions.ipynb


In [ ]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [ ]:
# your code here

## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [ ]:
# your code here

## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)

**Resources**:

- End of this code (`Additional summaries of topics and documents`) contains example of how to use `get_document_topics` and other steps to add topic probabilities back to data: https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/05_textasdata_partII_topicmodeling_solutions.ipynb
- If you're getting errors, use `shape`, `len`, and other commands to check the dimensionality of things at different steps 

In [ ]:
## your code here to get doc-level topic probabilities 

In [ ]:
## your code here to add those topic probabilities to the dataframe

In [ ]:
## your code here to summarize the topic proportions for each of the topics_clean 

# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [ ]:
## your code here 

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [ ]:
# your code here

# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [ ]:
# your code here 